##GOLD FACT: SALES


This notebook builds the Gold Sales fact table.
This is sales data that also shows who bought it and what they bought
Each row corresponds to one sales line item.


In [0]:
%sql
-- Preview CRM sales source data
SELECT *
FROM analytics.silver.crm_sales_details
LIMIT 10;


In [0]:
%sql
-- Preview Gold customers dimension
SELECT *
FROM analytics.gold.dim_customers
LIMIT 10;


In [0]:
%sql
-- Preview Gold products dimension
SELECT *
FROM analytics.gold.dim_products
LIMIT 10;


In [0]:
%sql
-- Create Gold fact table for sales
-- One row per sales transaction
-- Linked to customers and products dimensions

CREATE OR REPLACE TABLE analytics.gold.fact_sales AS
SELECT
    s.order_number                      AS order_id,
    s.order_date,
    s.ship_date,
    s.due_date,

    -- Foreign keys
    CAST(s.customer_id AS STRING)       AS customer_id,
    CAST(s.product_number AS STRING)    AS product_id,

    -- Measures
    s.quantity,
    s.price,
    s.sales_amount

FROM analytics.silver.crm_sales_details s;


In [0]:
%sql
-- Preview Gold fact table: sales
SELECT *
FROM analytics.gold.fact_sales
LIMIT 10;


## Sanity checks of gold table after writing

In [0]:
%sql
-- Sanity check: foreign keys should not be NULL
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customers,
    SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS null_products
FROM analytics.gold.fact_sales;


## Sanity Checks for Customers

Should not be NULL

In [0]:
%sql
-- Sanity check: one row per customer
SELECT customer_id, COUNT(*) AS cnt
FROM analytics.gold.dim_customers
GROUP BY customer_id
HAVING COUNT(*) > 1;


## For Product

In [0]:
%sql
-- Sanity check: one row per product
SELECT product_id, COUNT(*) AS cnt
FROM analytics.gold.dim_products
GROUP BY product_id
HAVING COUNT(*) > 1;
